In [1]:
from weather_data_api import get_predicted_data
from google_maps_api import fetch_roof_information, load_google_maps_api

import torch

In [3]:
df = get_predicted_data()
df.head()

INFO:weather_data_api:Fetching data from the KNMI API
INFO:weather_data_api:Fetching latest file of harmonie_arome_cy40_p1 version 0.2
INFO:weather_data_api:Latest file is: harm40_v1_p1_2024061100.tar
INFO:weather_data_api:Successfully downloaded dataset file to harm40_v1_p1_2024061100.tar
INFO:weather_data_api:Unpacking harm40_v1_p1_2024061100.tar
INFO:weather_data_api:Reading grib files and extracting data
INFO:weather_data_api:Renaming the columns and caching it


,temperature_sequence,global_irradiance_sequence,wind_speed_sequence,Q,dni_sequence,dhi_sequence
datetime,,,,,,
2024-06-11 00:00:00,10.196924,0.000000e+00,3.189144,-1987.476389,0.0,-1987.476389
2024-06-11 01:00:00,10.618799,5.228204e-11,3.340980,-9602.093333,0.0,-9602.093333
2024-06-11 02:00:00,10.822168,0.000000e+00,3.137340,-9458.431111,0.0,-9458.431111
2024-06-11 03:00:00,10.498926,0.000000e+00,3.050898,-5072.668889,0.0,-5072.668889
2024-06-11 04:00:00,9.864404,3.955200e+04,2.564199,-320.252406,0.0,-320.252406


In [4]:
load_google_maps_api()
roof_data = fetch_roof_information("51.425722,5.50894")
roof_data

{'azimuth': 110.63372, 'tilt': 34.84574}

In [5]:
df["azimuth"] = roof_data["azimuth"]
df["tilt"] = roof_data["tilt"]
df["module_type"] = "monocrystalline"

df.head()

,temperature_sequence,global_irradiance_sequence,wind_speed_sequence,Q,dni_sequence,dhi_sequence,azimuth,tilt,module_type
datetime,,,,,,,,,
2024-06-11 00:00:00,10.196924,0.000000e+00,3.189144,-1987.476389,0.0,-1987.476389,110.63372,34.84574,monocrystalline
2024-06-11 01:00:00,10.618799,5.228204e-11,3.340980,-9602.093333,0.0,-9602.093333,110.63372,34.84574,monocrystalline
2024-06-11 02:00:00,10.822168,0.000000e+00,3.137340,-9458.431111,0.0,-9458.431111,110.63372,34.84574,monocrystalline
2024-06-11 03:00:00,10.498926,0.000000e+00,3.050898,-5072.668889,0.0,-5072.668889,110.63372,34.84574,monocrystalline
2024-06-11 04:00:00,9.864404,3.955200e+04,2.564199,-320.252406,0.0,-320.252406,110.63372,34.84574,monocrystalline


In [6]:
dynamic_cols = [
    "temperature_sequence",
    "wind_speed_sequence",
    "dni_sequence",
    "dhi_sequence",
    "global_irradiance_sequence",
]
static_cols = ["tilt", "azimuth", "module_type"]

module_type_map = {
    "monocrystalline": 0,
    "polycrystalline": 1,
}

dynamic_sample = torch.tensor(df[dynamic_cols].values, dtype=torch.float32)

In [9]:
# # Convert into two batches of size 24x5
dynamic_sample_reshaped = dynamic_sample.view(-1, 24, 5)
dynamic_sample_reshaped.shape

torch.Size([2, 24, 5])

In [14]:
# static_sample must be in the format Bx3
# the three is for the three static features: tilt, azimuth, module_type
# module_type is a categorical feature, so it must be label encoded

mapped_df = df.copy()
mapped_df["module_type"] = mapped_df["module_type"].map(module_type_map)
static_sample = torch.tensor(mapped_df[static_cols].values, dtype=torch.float32)

# # Convert into two batches of size 1x3
static_sample_reshaped = static_sample.view(-1, 3)[:2]

static_sample_reshaped.shape

torch.Size([48, 3])